## Libraries

In [1]:
!pip3 install numpy
!pip3 install pandas
!pip3 install numpy
!pip3 install sklearn
!pip3 install statsmodels
!pip install matplotlib

import matplotlib.pyplot as plt

import sklearn
import time
import random
import copy
import json

import pandas as pd # data manipulation library
import numpy as np # math library

import sklearn.metrics as sklm # metrics
import skll.metrics as skllm
import statsmodels as sm # statistical models
from sklearn.model_selection import GridSearchCV
from sklearn import svm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Utils

In [2]:
PATH = "./"

In [3]:
class WalkingForwardTimeSeriesSplit():
  def __init__(self, n_splits):
    self.n_splits = n_splits
  
  def get_n_splits(self, X, y, groups):
    return self.n_splits
  
  def split(self, X, y=None, groups=None):
    n_samples = len(X)
    k_fold_size = n_samples // self.n_splits
    indices = np.arange(n_samples)

    margin = 0
    for i in range(self.n_splits):
      start = i * k_fold_size
      stop = start + k_fold_size
      mid = int(0.8 * (stop - start)) + start
      yield indices[start: mid], indices[mid + margin: stop]

In [4]:
def clean_cv_results(cv_results):
  res = copy.deepcopy(cv_results)
  for key in res:
    if type(res[key]) != list: 
      res[key] = res[key].tolist()

  return res

In [5]:
def retrieve_data(flow_interval):
    path = "{0}dataset/dataset_flow_{1}.csv".format(PATH, flow_interval)
    print(PATH)
    data = pd.read_csv(path, ';')
    
    data['Flow'].apply(int)
    data['AveSpeed'].apply(float)
    data['Density'].apply(float)
    data['Sunday'].apply(int)
    data['Monday'].apply(int)
    data['Tuesday'].apply(int)
    data['Wednesday'].apply(int)
    data['Thursday'].apply(int)
    data['Friday'].apply(int)
    data['Saturday'].apply(int)
      
    return data

In [6]:
def print_json (obj):
  print(json.dumps(obj, sort_keys=True, indent=4))

In [7]:
def store(obj, path, name):
  with open("{0}{1}/{2}.json".format(PATH, path, name), 'w') as json_file:
    json.dump(obj, json_file, sort_keys=True, indent=4)

In [8]:
def store_results ():
  name = int(time.time())
  
  result_data['meta'] = {
    "SEEABLE_PAST": SEEABLE_PAST,
    "PREDICT_IN_FUTURE": PREDICT_IN_FUTURE,
    "FLOW_INTERVAL": FLOW_INTERVAL,
    "N_SPLITS": N_SPLITS,
  }

  store(result_data, "results", name)

  slim_result_data = copy.deepcopy(result_data)
  for model in slim_result_data['results']:
      del slim_result_data['results'][model]['raw']

  store(slim_result_data, "results", "{0}_slim".format(name))

In [9]:
def store_comparisons (title):
  name = str(int(time.time()))
  
  j = copy.deepcopy(comparison_data)

  store(j, "results/comparison", "{0}_{1}".format(title, name))
    
  for i in range(len(j)):
    for model in j[i]['results']:
      del j[i]['results'][model]['raw']

  store(j, "results/comparison", "{0}_{1}_slim".format(title, name))

In [10]:
def generate_dataset(data, useB, n_steps, n_future):
  """ Generate Dataset
  
  Generate a dataset provided a sequence. Reshape the sequence in rolling intervals from [samples, timesteps] into 
  [samples, timesteps, features] and split the sequence. The split the sequence in rolling intervals with a corresponding value 
  like the example bellow.

  Ex: split_sequence([1, 2, 3, 4, 5], 3) #([[1, 2, 3], [2, 3, 4]], [4, 5])
  
  Arguments:
    raw_seq: the sequence to reshape.
    useB: if the dataset is more complex or not.
    n_steps: size of the rolling interval
    n_future: the distance to the interval the value should be.  
  """

  sequence = np.array(data if useB else data['Flow'])

  n = len(sequence)
  X, Y = list(), list()

  for i in range(n):
    j = i + n_steps
    k = j + n_future

    if k >= n:
      break

    seq_x, seq_y = sequence[i:j], sequence[k]
    X.append(seq_x)	
    Y.append(seq_y[0] if useB else seq_y)

  X, Y = np.array(X), np.array(Y)	
  
  if not useB:
    X = X.reshape((X.shape[0], X.shape[1], 1))

  return X, Y

In [11]:
def evaluate (expected, observed, times, name):
 
  n = len(expected)
  flatten = lambda l : [i for sl in l for i in sl]
  
  expected = list(map(list, expected))
  observed = list(map(list, observed))
  
  for i in range(n):
    expected[i] = list(map(float, expected[i]))
    observed[i] = list(map(float, observed[i]))
  
  raw = evaluate_raw(expected, observed, times)
  
  eva = {
    'TIME':         int(sum(times)),
    'RMSE':         float(np.mean(raw['RMSE'])),
    'MAE':          float(np.mean(raw['MAE'])),
    'Kappa':        float(np.mean(raw['Kappa'])),
    'HR':           float(np.mean(raw['HR'])),
    'has_negative': (min(flatten(observed)) < 0),
    'raw':          raw
  }
  
  print(f"\n{name} Final Result:")
  print(f"\tTotal Time: {eva['TIME']}s")
  print(f"\tRMSE: {eva['RMSE']}")
  print(f"\tMAE: {eva['MAE']}")
  print(f"\tKappa: {eva['Kappa']}")
  print(f"\tHit Ratio: {eva['HR'] * 100}%")
    
  return eva

In [23]:
def evaluate_raw (expected, observed, times):
  
  n = len(expected)

  for i in range(n):
    observed[i] = [0 if np.isnan(o) else o for o in observed[i]]

  for i in range(n):
    observed[i] = [max(o, 0) for o in observed[i]]
  
  raw = {
    'expected': expected,
    'observed': observed,
    'TIME':     times,
    'RMSE':     [0] * n,
    'MAE':      [0] * n,
    'Kappa':    [0] * n,
    'HR':       [0] * n,
  }
  
  for i in range(n):
    Y     = expected[i]
    Y_hat = observed[i]
    time  = times[i]

    raw['RMSE'][i]  = np.sqrt(sklm.mean_squared_error(Y, Y_hat))
    raw['MAE'][i]   = sklm.mean_absolute_error(Y, Y_hat)
    raw['Kappa'][i] = skllm.kappa(Y, Y_hat)
    raw['HR'][i]    = evaluate_precision_hit_ratio(Y, Y_hat)
    
    if VERBOSITY:
      print(f"({i+1}/{n}) Test Size: {len(Y)}, Time: {time}s")
      print(f"\tRMSE: {raw['RMSE'][i]}")
      print(f"\tMAE: {raw['MAE'][i]}")
      print(f"\tKappa: {raw['Kappa'][i]}")
      print(f"\tHit Ratio: {raw['HR'][i] * 100}%")

  return raw

In [13]:
def evaluate_precision_hit_ratio (Y, Y_hat):
  
  cnt = 0
  
  for i in range(len(Y)):
    if i < N_FUTURE:
      continue
      
    exp = Y[i] - Y[i - N_FUTURE]
    obs = Y_hat[i] - Y[i - N_FUTURE]
    
    if exp * obs > 0:
      cnt += 1
    
  return cnt / len(Y)

## Testing

In [14]:
# Model Parameters
SEEABLE_PAST = 480 # in minutes
PREDICT_IN_FUTURE = 60 # in minutes
FLOW_INTERVAL = 150 # the interval size for each flow
N_SPLITS = 8

In [15]:
# Derivated Model Parameters
N_STEPS = SEEABLE_PAST * 60 // FLOW_INTERVAL # the number of flows to see in the past
N_FUTURE = PREDICT_IN_FUTURE * 60 // FLOW_INTERVAL # how much in the future we want to predict (0 = predict the flow on the next FLOW_INTERVAL minutes)
DAY_SIZE = (24 * 60 * 60) // FLOW_INTERVAL  
WEEK_SIZE = (7 * 24 * 60 * 60) // FLOW_INTERVAL
VERBOSITY = True

In [16]:
def get_svm_tuned(X, Y, useB):
  param_grid = {
    'C': [1.0, 10.0, 100.0, 1000.0],
    'gamma': list(np.logspace(-2, 2, 4)) + ['scale'],
  }
  model = svm.SVR(epsilon=0.2)
  scoring = 'neg_mean_squared_error'
  cv = WalkingForwardTimeSeriesSplit(n_splits=1)
  n_jobs = 15

  grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=cv, n_jobs=n_jobs, verbose=2)

  grid_search.fit(X, Y)
    
  res = {
      'params': param_grid,
      'best_params': grid_search.best_params_,
      'score': clean_cv_results(grid_search.cv_results_),
  }

  return grid_search.best_estimator_, res

In [17]:
def support_vector_machine(X, Y, useB=False,):
  global result_data
  
  X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])

  name = "SVM B" if useB else "SVM A"
  expected, observed, times, grid_res = [], [], [], []
  tscv = WalkingForwardTimeSeriesSplit(n_splits=N_SPLITS)

  for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    start_time = time.time()

    model, res = get_svm_tuned(X_train, Y_train, useB)
    model.fit(X_train, Y_train) 

    end_time = time.time()
    
    grid_res.append(res)
    observed.append(model.predict(X_test))
    expected.append(Y_test)
    times.append(end_time - start_time)
    
  res = evaluate(expected, observed, times, name)  

  res['grid_res'] = grid_res

  result_data['results'][name] = res

  store(result_data['results'][name], "results/grid", "{0}_{1}".format(name, PREDICT_IN_FUTURE))

In [18]:
global result_data
  
result_data = {
    'results': {},
    'meta': {
      'SEEABLE_PAST': SEEABLE_PAST,
      'PREDICT_IN_FUTURE': PREDICT_IN_FUTURE,
      'FLOW_INTERVAL': FLOW_INTERVAL,
      'N_SPLITS': N_SPLITS,
    }
}

data = retrieve_data(FLOW_INTERVAL)
X_a, Y_a = generate_dataset(data, False, N_STEPS, N_FUTURE)
support_vector_machine(X_a, Y_a)

./
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
(1/8) Test Size: 1320, Time: 68.24463868141174s
	RMSE: 4.0789799283383354
	MAE: 2.9463283624753536
	Kappa: 0.13088904403091084
	Hit Ratio: 67.87878787878789%
(2/8) Test Size: 1320, Time: 61.9749698638916s
	RMSE: 3.792278696137758
	MAE: 2.7784718155865846
	Kappa: 0.08975887845030683
	Hit Ratio: 61.66666666666667%
(3/8) Test Size: 1320, Time: 61.1627151966095s
	RMSE: 4.3040327503933895
	MAE: 3.1510773620931545
	Kappa: 0.09740509353231341
	Hit Ratio: 65.75757575757576%
(4/8) Test Size: 1320, Time: 57.629284

In [19]:
def plot_prediction (Y, Y_hat, title):
  """ Plot Prediction
  
  Plot the prediction (Flow x Time) of what was expected and what
  was predicted.
  """

  for i in range(len(Y)):
    name = f"{title} ({str(i+1).zfill(2)} of {len(Y)})"
    path = f"plots/prediction/{name}"
    
    plt.plot(Y[i])
    plt.plot(Y_hat[i])
    plt.title(f"Predição do Modelo {title}")
    plt.ylabel('Fluxo')
    plt.xlabel('Tempo')
    plt.legend(['esperado', 'observado'], loc='upper left')
    plt.rcdefaults()

    plt.savefig(path + ".png", bbox_inches='tight')
    # plt.savefig(path + ".pdf")

    plt.close('all')

In [20]:
expected = result_data['results']['SVM A']['raw']['expected']
observed = result_data['results']['SVM A']['raw']['observed']

plot_prediction(expected, observed, "SVM")